## Project Goal

---

### The objective of this analysis is to evaluate economic performance of European regions by identifying:
- which regions grow the fastest,
- which regions demonstrate the most stable development,
- and which regions provide the best balance between growth, stability, and economic scale.

### To achieve this, we construct a composite economic score combining:
- CAGR (long-term growth),
- growth volatility (economic stability),
- and average GDP size (economic weight).

In [ ]:
%load_ext autoreload
%autoreload 2

import sys
from pathlib import Path

PROJECT_ROOT = Path().resolve().parent
sys.path.append(str(PROJECT_ROOT))

In [ ]:
from ingestion.db_engine import get_engine

engine = get_engine()

engine

In [ ]:
from sqlalchemy import text

with engine.connect() as conn:
    db = conn.execute(text("SELECT current_database();")).fetchone()
    print("Connected to:", db[0])

In [ ]:
import pandas as pd

df = pd.read_sql("SELECT * FROM public.regional_gdp", con=engine)

df.head()

In [ ]:
df.info()
df.describe()
df.isna().sum()

In [ ]:
assert not df.duplicated(["geo_code","year"]).any()

In [ ]:
df = df.sort_values(["geo_code", "year"])

In [ ]:
cagr = (
    df.groupby("geo_code")
    .agg(
        start_gdp=("gdp_mio_eur", "first"),
        end_gdp=("gdp_mio_eur", "last"),
        start_year=("year", "min"),
        end_year=("year", "max"),
    )
)

cagr["years"] = cagr["end_year"] - cagr["start_year"]
cagr["CAGR"] = ((cagr["end_gdp"] / cagr["start_gdp"]) ** (1 / cagr["years"]) - 1)

cagr = cagr.reset_index()

In [ ]:
volatility = (
    df.groupby("geo_code")
      .agg(
          avg_gdp=("gdp_mio_eur", "mean"),
          std_gdp=("gdp_mio_eur", "std")
      )
      .reset_index()
)

avg_gdp = df.groupby("geo_code")["gdp_mio_eur"].mean().reset_index()

volatility["coef_var"] = (
    volatility["std_gdp"] / volatility["avg_gdp"]
)

In [ ]:
top10 = (
    cagr
    .sort_values("CAGR", ascending=False)
    .head(10)
    .copy()
)

top10["CAGR_percent"] = (top10["CAGR"] * 100).round(3)

top10[["geo_code", "start_year", "end_year", "CAGR_percent"]]

In [ ]:
vol = (
    volatility
    .head()
    .copy()
)

vol[["geo_code", "std_gdp", "avg_gdp", "coef_var"]].round(2)

In [ ]:
import numpy as np

metrics = (
    cagr[["geo_code", "CAGR"]]
    .merge(volatility[["geo_code", "coef_var"]], on="geo_code")
    .merge(avg_gdp[["geo_code", "gdp_mio_eur"]], on="geo_code")
)

metrics = metrics.rename(columns={
    "gdp_mio_eur": "avg_gdp",
    "coef_var": "volatility"
})

In [ ]:
metrics.head()

In [ ]:
def minmax(series):
    return (series - series.min()) / (series.max() - series.min())

metrics["normalized_CAGR"] = minmax(metrics["CAGR"])
metrics["normalized_avg_gdp"] = minmax(metrics["avg_gdp"])
metrics["normalized_volatility"] = minmax(metrics["volatility"])

In [ ]:
def compute_score(df: pd.DataFrame, w_growth: float, w_vol: float, w_size: float) -> float:
    score = (
        w_growth * df["normalized_CAGR"]
        - w_vol * df["normalized_volatility"]
        + w_size * df["normalized_avg_gdp"]
    )
    return score

In [ ]:
weight_scenarios = {
    "growth_focus": (0.7, 0.2, 0.1),
    "balanced": (0.5, 0.3, 0.2),
    "stability_focus": (0.4, 0.5, 0.1),
    "size_focus": (0.4, 0.2, 0.4),
}

In [ ]:
rankings = {}

for name, (wg, wv, ws) in weight_scenarios.items():
    tmp = metrics.copy()

    tmp["score"] = compute_score(tmp, wg, wv, ws)

    tmp = (
        tmp.sort_values("score", ascending=False)
           .reset_index(drop=True)
    )

    tmp["rank"] = tmp.index + 1

    rankings[name] = tmp[["geo_code", "rank", "score"]]

In [ ]:
top_regions = {}

for name, df_rank in rankings.items():
    top_regions[name] = set(df_rank.head(10)["geo_code"])

top_regions

In [ ]:
common_top = set.intersection(*top_regions.values())
common_top

In [ ]:
rank_compare = rankings["balanced"][["geo_code", "rank"]].rename(
    columns={"rank": "balanced_rank"}
)

for name in weight_scenarios:
    rank_compare = rank_compare.merge(
        rankings[name][["geo_code", "rank"]]
        .rename(columns={"rank": f"{name}_rank"}),
        on="geo_code"
    )

rank_compare = rank_compare.rename(columns={
    "balanced_rank_x": "rank_old",
    "balanced_rank_y": "rank_new"
})

rank_compare.head()

In [ ]:
import plotly.express as px
import plotly.io as pio
pio.templates.default = "plotly_dark"

fig = px.scatter(
    rank_compare,
    x="rank_old",
    y="rank_new",
    text="geo_code",
    title="Ranking Change (Before vs After Composite Score)"
)

fig.update_traces(textposition="top center")
fig.show()

In [ ]:
fig = px.line(
    df,
    x="year",
    y="gdp_mio_eur",
    color="geo_code",
    title="Regional GDP dynamics (EU regions)"
)

fig.show()

In [ ]:
fig = px.bar(
    cagr.sort_values("CAGR", ascending=False),
    x="geo_code",
    y="CAGR",
    title="GDP CAGR by Region"
)

fig.show()

In [ ]:
avg_gdp = (
    df.groupby("geo_code")["gdp_mio_eur"]
      .mean()
      .reset_index()
      .sort_values("gdp_mio_eur", ascending=False)
)

fig = px.bar(
    avg_gdp.head(15),
    x="geo_code",
    y="gdp_mio_eur",
    title="Top regions by average GDP"
)

fig.show()

### Key Insights

**Which regions grow fastest?**
Regions such as **FIZ, RO11, AT0, RO3, ITT0, CZ02, and PL91** demonstrate the highest CAGR values, indicating rapid economic expansion over the observed period.

**Which regions grow most consistently?**
Regions including **AL01, AT11, and AT22** show smooth and stable GDP trajectories with minimal volatility, suggesting steady and predictable growth.

**Which regions combine growth and stability?**
Regions like **AT0, CZ02, and PL91** achieve a strong balance between above-average growth and relatively stable development patterns.

**Sensitivity analysis** shows that regions remain stable in their positions under various weighting schemes, indicating stable economic indicators that do not depend on model assumptions.

---

### Recommendations

- **High growth but risky:** *FIZ, RO11* — strong expansion but likely higher volatility and sensitivity to economic shocks.
- **Stable investment candidate:** *AT11, AT22* — moderate but consistent growth, suitable for long-term stability-focused strategies.